# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from os import listdir

## Creacion de indice para recorrer instancias

In [2]:
##ESTO ES DE UN TP ANTERIOR, NO ME ACUERDO BIEN PARA QUE ESTABA
##ESTO ES MEDIO FEO PERO ANDA, NO TOCAR##

filas_indice = []

instancias = listdir('./instancias')
instancias = sorted(instancias)

i = 6

tipo = "IN"
for instancia in instancias:  
    if 'CMI' in instancia:
        if i > 26:
            n = int(F"{instancia[5]}")
        else:
            n = int(F"{instancia[5]}{instancia[6]}")
            
        filas_indice.append([instancia, n, tipo])
        if tipo == "IN":
            tipo = "OUT"
        else:
            i = i+2
            tipo = "IN"
filas_indice.sort(key=lambda x: x[1])
pd.DataFrame(filas_indice, columns=["instancia", "n", "IN/OUT"]).to_csv("./instancias/indice.csv", index=False, header=True)

A continuación leemos los datasets en dataframes de Pandas.

In [3]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read()


df_todas = pd.read_csv("./instancias/indice.csv")

df_todas_in = df_todas[df_todas["IN/OUT"] == "IN"]
df_todas_out = df_todas[df_todas["IN/OUT"] == "OUT"]


La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.


In [4]:
def correr_experimento(archivo_entrada, archivo_salida, metodo):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./build/tp1", archivo_entrada, archivo_salida, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar y leer salida de STDERR con el tiempo de ejecución.
    clog = process.communicate()[1] # communicate() devuelve una tupla (stdout, stderr)

    tiempo_de_ejecucion = str.splitlines(clog)[0]
    #print(clog)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion con el metodo {'EG' if metodo == 0 else 'LU'} sobre la instancia {archivo_entrada}.")
    return float(tiempo_de_ejecucion)

Creadores de Instancias

In [5]:
def instancia_uniforme(numDeAngulos, iso):
    return np.random.uniform(0, iso-1, numDeAngulos)

def instancia_normal(numDeAngulos, iso):
    temperaturasExternas = np.zeros(numDeAngulos)
    numDeTemperaturas = 0
    while numDeTemperaturas < numDeAngulos :
        temperaturaRandom = np.random.normal(iso/2, 100)
        if temperaturaRandom >= 0 and temperaturaRandom < iso : 
            temperaturasExternas[numDeTemperaturas] = temperaturaRandom
            numDeTemperaturas = numDeTemperaturas +1
    return temperaturasExternas

def crear_archivo_con_instacia (tipoDeInstancia, nombreArchivo, paredInterna, paredExterna, numDeAngulos, numDeRadios, iso, numDeInstancias):
    
    with open("./instancias/" + nombreArchivo, 'w') as f: #abrimos el archivo
        #escribimos los datos del principio del archivo
        f.write(str(paredInterna)+" "+str(paredExterna)+" "+str(numDeAngulos)+" "+str(numDeRadios)+" "+str(iso)+" "+str(numDeInstancias)+"\n")
        #creamos las temperaturas externas y las escribimos junto con las temperaturas internas
        for i in range(0, numDeInstancias):

            instancias = np.empty(numDeAngulos)
            if tipoDeInstancia == 0 :
                instancias = instancia_uniforme(numDeAngulos, iso)
            elif tipoDeInstancia == 1 :
                instancias = instancia_normal(numDeAngulos, iso)
            else:
                break

            for j in range(0, numDeAngulos) :
                f.write("1500 ")
            for temperatura in instancias :
                f.write(str(temperatura)+" ")
            f.write("\n")


## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [14]:

#Guardar aquí un vector [archivo_entrada, archivo_salida, metodo (0 o 1)] por cada experimento a correr
def cargarExperiementos(tipoDeInstancia, variante):
    exps=[]
    instancias = listdir(f"instancias/{tipoDeInstancia}")
    if variante == "EG":
        exps = [[f"{instancia}", f"${instancia}-out", 0] for instancia in instancias]
    elif variante == "LU":
        exps = [[f"{instancia}", f"${instancia}-out", 1] for instancia in instancias]
    elif variante == "ALL":
        exps = cargarExperimentos(tipodeInstancia, "EG").append(cargarExperimentos(tipodeInstancia, "LU"))
    return exps

In [ ]:
experimentos = [];

## Experimento 1



In [ ]:
experimentos.append(["./instancias/testInput.txt", "./resultados/testOutput.txt", "0"])

## Experimento 2



## Experimento 3



## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [ ]:
columnas = ["entrada", "salida", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []    
    for i in range(0, T):
        tiempo = correr_experimento(experimento[0], experimento[1], experimento[2])
        tiempos.append(tiempo);

    tiempo_medio = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_medio]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("./resultados/resultado.csv", index=False, header=True);

'Experimento: 1/1'